In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from numpy                 import array
from sklearn               import metrics
from sklearn.preprocessing import MinMaxScaler
from keras.models          import Sequential
from keras.layers          import *
from sklearn               import metrics
from tensorflow.keras import layers
from tcn import TCN
import keras
from keras.layers import Conv1D, MaxPooling1D, Dense, Flatten,Conv2D, MaxPool2D,LSTM,Bidirectional
from keras.models import Sequential
from keras import Input
from keras.layers          import *
from keras.models import save_model,load_model
from numpy import save,load
from sklearn.model_selection import KFold
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 忽略 INFO 和 WARNING 信息
tf.get_logger().setLevel('ERROR')  # 只显示 ERROR 信息

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
"""
MSE  ：均方误差    ----->  预测值减真实值求平方后求均值
RMSE ：均方根误差  ----->  对均方误差开方
MAE  ：平均绝对误差----->  预测值减真实值求绝对值后求均值
R2   ：决定系数，可以简单理解为反映模型拟合优度的重要的统计量
"""
def compute_metrics(pred,real):
    MSE   = metrics.mean_squared_error(pred, real)
    RMSE  = metrics.mean_squared_error(pred, real)**0.5
    MAE   = metrics.mean_absolute_error(pred, real)
    R2    = metrics.r2_score(pred, real)
    print('均方误差: %.5f' % MSE)
    print('均方根误差: %.5f' % RMSE)
    print('平均绝对误差: %.5f' % MAE)
    print('R2: %.5f' % R2)
    return [MSE,RMSE,MAE,R2]


In [ ]:
##dscribe 1.2.1 numpy 1.23.5
def load_trained_model(model_type, saved_model_path):
    # Load the model
    if model_type in [7, 8]:  # For TCN models
        #laptop running have to change to complie=False and tf....
        current_model = tf.keras.models.load_model(saved_model_path, custom_objects={'TCN': TCN},compile=False)
    else:
        current_model = tf.keras.models.load_model(saved_model_path,compile=False)
    return current_model

In [ ]:
def batch_predict(model_type, model, data, batch_size=16):
    """ 对数据进行分批预测，优化内存使用。 """
    n_samples = data.shape[0]
    sample_prediction = model.predict(data[:1], verbose=0)

    # 检查模型的返回类型，并据此获取形状
    if isinstance(sample_prediction, list):
        # 如果模型返回的是一个列表（多输出模型），取第一个输出的形状
        pred_shape = sample_prediction[0].shape
    else:
        # 否则直接取返回值的形状
        pred_shape = sample_prediction.shape

    # 根据model_type进行特殊处理
    if model_type in [8, 9]:
        predictions = np.zeros((n_samples, pred_shape[1]), dtype='float16')  # 针对8和9的特殊预分配结果数组
    else:
        predictions = np.zeros((n_samples, *pred_shape[1:]), dtype='float16')  # 通常情况下的预分配结果数组

    for start in range(0, n_samples, batch_size):
        end = min(start + batch_size, n_samples)
        batch_data = data[start:end]
        batch_pred = model.predict(batch_data, verbose=0)

        # 根据model_type进行特殊处理
        if model_type in [8, 9]:
            batch_pred = batch_pred.reshape(batch_pred.shape[0], batch_pred.shape[1])

        predictions[start:end] = batch_pred

        del batch_data  # 删除不再需要的批次数据
        gc.collect()  # 调用垃圾回收

    return predictions

In [ ]:
def compute_MAE_within_percent(pred_test,real_test,pred_train,real_train,threshold):
    difference_test=abs(pred_test-real_test)
    difference_train=abs(pred_train-real_train)
    i_test=0
    i_train=0
    for each in difference_test:
        if each<threshold:
            i_test+=1
    for each in difference_train:
        if each<threshold:
            i_train+=1
    percent_train=i_train/len(difference_train)
    percent_test=i_test/len(difference_test)
    return percent_train,percent_test

In [ ]:
import os
import tensorflow as tf
from sklearn.model_selection import KFold
import joblib
import gc
def evaluate_model_in_cv(predict_target, current_model, model_type, X_cv, y_cv, Outputscaler, start_fold=0, end_fold=10):
    print('now model type is', model_type)
    cvscores_TEST = []
    kfold = KFold(n_splits=10, shuffle=True, random_state=1)
    
    for fold_number, (_, test) in enumerate(kfold.split(X_cv, y_cv)):
        print(f"Fold {fold_number + 1} of {kfold.get_n_splits()}")
        if fold_number < start_fold or fold_number >= end_fold:
            continue

        ##### 给出预测 #####
        try:
            pred_test = batch_predict(model_type, current_model, X_cv[test])
            pred_test = pred_test.reshape(-1, 1)
            pred_test = Outputscaler.inverse_transform(pred_test)
            real_test = Outputscaler.inverse_transform(y_cv[test].reshape(-1, 1))

            TEST_SCORES = compute_metrics(pred_test, real_test)

            ### 测试分数 ###
            cvscores_TEST.append(TEST_SCORES)
        except Exception as e:
            print(e)

        print(TEST_SCORES)

    # 计算平均值和标准差
    average_scores_TEST, std_scores_TEST = calculate_statistics(cvscores_TEST)
    print(average_scores_TEST, std_scores_TEST)
    return average_scores_TEST, std_scores_TEST

# 计算平均值和标准差的辅助函数
def calculate_statistics(scores):
    averages = [sum(score) / len(score) for score in zip(*scores)]
    std_devs = [(sum((x - avg) ** 2 for x in score) / len(score)) ** 0.5 for avg, score in zip(averages, zip(*scores))]
    return averages, std_devs


In [ ]:
import pandas as pd
def evaluate_all_combinations():
    targets = ["SLAB", "O", "OH", "OOH"]
    model_sources = ["A_B", "A", "B"]
    model_types = list(range(1, 11))
    dataset_map = {"A": "B", "B": "A", "A_B": "A"}  # 模型来源对应的数据集,A to B, Bto A and A_B to B

    # 创建一个空的DataFrame来存储结果
    results_df = pd.DataFrame()

    for target in targets:
        for source in model_sources:
            for model_type in model_types:
                # A_B模型缺少2和4
                if source == "A_B" and model_type in [2, 4]:
                    continue
                # 根据模型来源确定数据集
                dataset = dataset_map[source]
                print("current target:",target,"current source:",source,"current model_type:" ,model_type,"current dataset:",dataset)
                # 构建文件路径
                model_path = f"./{source} models/{source}_{target}_{model_type}_best_model.h5"
                input_path = f"./{target}_INPUT_{dataset}_SCALED.npy"
                output_path = f"./{target}_OUTPUT_{dataset}_SCALED.npy"
                scaler_path = f"./{target}_OUTPUT_SCALER_{dataset}.pkl"

                # 加载模型和数据
                model = load_trained_model(model_type, model_path)
                X_scaled = np.load(input_path)
                y_scaled = np.load(output_path)
                if model_type==10:
                    X_scaled = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], X_scaled.shape[2], 1)

                output_scaler = joblib.load(scaler_path)

                # 评估模型
                average_scores_TEST, std_scores_TEST = evaluate_model_in_cv(target, model, model_type, X_scaled, y_scaled, output_scaler)

                # 准备要添加到DataFrame的数据
                result_data = {
                    "Target": target,
                    "Model Source": source,
                    "Model Type": model_type,
                    "Dataset": dataset
                }
                for i, (avg_score, std_score) in enumerate(zip(average_scores_TEST, std_scores_TEST)):
                    result_data[f"Average Score {i+1}"] = avg_score
                    result_data[f"Standard Deviation {i+1}"] = std_score

                # 将结果添加到DataFrame
                results_df = results_df.append(result_data, ignore_index=True)

    # 保存结果到CSV文件
    results_df.to_csv("evaluation_results.csv", index=False)

    return results_df

# 调用函数
results_df = evaluate_all_combinations()